In [1]:
import cga_utils
import prompt_versions
%load_ext autoreload
%autoreload 2

In [2]:
from langchain.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.s2_1.db"))

In [3]:
from langchain_community.chat_models import ChatOllama
#llm = ChatOllama(model="qwen3:4b", temperature = 0.0, top_p = 1, repeat_penalty=1, presence_penalty=0, frequency_penalty=0)  # vagy bármely más elérhető Ollama model
llm = ChatOllama(model="hf.co/unsloth/Qwen3-4B-Instruct-2507-GGUF:latest")  # vagy bármely más elérhető Ollama modell

In [71]:
dataset_tr = pd.read_json('dataset_raw/tatqa_dataset_train.json')    
dataset_t = pd.read_json('dataset_raw/tatqa_dataset_test.json')    
dataset_tg = pd.read_json('dataset_raw/tatqa_dataset_test_gold.json')    
dataset_d = pd.read_json('dataset_raw/tatqa_dataset_dev.json')    

sample_df = dataset_tg.sample(frac=0.20, random_state=42)
#sample_df = dataset_tr.sample(frac=0.031, random_state=42)


In [72]:
len(dataset_tr), len(dataset_d),  len(dataset_t), len(dataset_tg), len(sample_df)

(2201, 278, 278, 277, 55)

In [78]:
for i, item in sample_df.iterrows():    
    table = item['table']['table']     
    for q_item in item['questions']:
        for idev, itemd in dataset_d.iterrows():
            tabled = itemd['table']['table']     
            for q_itemd in itemd['questions']: 
                if q_itemd['answer_type'] == "arithmetic" and q_item['question'] == q_itemd['question']:
                #if q_item['question'] == q_itemd['question']:
                    print('duplicate', q_item['question'], q_itemd['question'], q_item['answer'],  q_itemd['answer'])
                

In [79]:
import pandas as pd
messages = prompt_versions.prompt_versions['V18'] #Base
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770204294.33221


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.4
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 3.26
<< ((3,522/79,828)) * 100 || [3522.0, 107962.0, 3.262258942961412] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.0, -5410.0, -39

In [80]:
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('res/Ollama_2_gemma3n_e4b_code_values_v11_m1.csv')
cga_utils.calc_overall_em(annotated_results)

0.7362637362637363

In [76]:
import pandas as pd
messages = prompt_versions.prompt_versions['V29'] #Best alg
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770165553.0563605


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 18.13
<< ((3,522/79,828)) * 100 || [3522.0, 29920.0, 10493.0, 19427.0, 18.12940752560869] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-

In [77]:
annotated_results = cga_utils.annotate_results(res)
#annotated_results.to_csv('res/Ollama_2_gemma3n_e4b_code_values_v11_m1.csv')
cga_utils.calc_overall_em(annotated_results)

0.7472527472527473

In [81]:
import pandas as pd
messages = prompt_versions.prompt_versions['V38b']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770226816.8635337


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 2.97
<< ((3,522/79,828)) * 100 || [3522.0, 118455.0, 2.973280992782069] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.

In [82]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.6923076923076923

In [83]:
import pandas as pd
messages = prompt_versions.prompt_versions['V23']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770226917.6007342


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 3.26
<< ((3,522/79,828)) * 100 || [3522.0, 107962.0, 3.262258942961412] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.

In [84]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.6923076923076923

In [85]:
import pandas as pd
messages = prompt_versions.prompt_versions['V41']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770227061.2707314


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.4
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 3.26
<< ((3,522/79,828)) * 100 || [3522.0, 107962.0, 3.262258942961412] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.0, -5410.0, -39

In [86]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.7362637362637363

In [87]:
import pandas as pd
messages = prompt_versions.prompt_versions['V43c']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770227201.5165617


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 11.77
<< ((3,522/79,828)) * 100 || [3522.0, 29920.0, 10493.0, 29920.0, 11.77139037433155] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-

In [88]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.7472527472527473

In [89]:
import pandas as pd
messages = prompt_versions.prompt_versions['V31']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770227410.89154


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 3.26
<< ((3,522/79,828)) * 100 || [3522.0, 107962.0, 3.262258942961412] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.

In [90]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.7362637362637363

In [91]:
import pandas as pd
messages = prompt_versions.prompt_versions['V44b']
#messages = [(a, t.replace('/no_think','')) for (a,t)  in messages]
#messages[-1] = ( messages[-1][0],  messages[-1][1] + "/no_think" ) 

res = cga_utils.execute_dataset_predictions(llm, messages, dataset=sample_df)

1770227556.3287506


0it [00:00, ?it/s]

7e2290f656cbeef6d9a61c02767b752c
What was the percentage change in adjusted EBITDA between 2018 and 2019? failure: 16.36 good answer:  16.36 
<< ((108,307-93,081)/93,081) * 100 || [108307.0, 93081.0, 16.357795898196194] || [108307.0, 93081.0, 93081.0, 100.0] || False >>
*** Overall EM:  1.0 / 30
d2fca279eeba2f76f54127d5884fc542
What is the change in revenue growth between 2019 and 2018? Success: -4.3999999999999995
<< ((7.2%-11.6%)) * 100 || [7.2, 11.6, -4.3999999999999995] || [7.2, 11.6] || True >>
15c13aff533786d03238eb3e6f433c1f
What are leasehold improvements in 2019 as a percentage of the total property equipment before depreciation, amortization and fixed assets not in service? Success: 2.97
<< ((3,522/79,828)) * 100 || [3522.0, 118455.0, 2.973280992782069] || [3522.0, 79828.0] || False >>
*** Overall EM:  0.6666666666666666 / 220
a232d237c226f8bcfdabfd8daad919fc
What was the change in Property and equipment, net in 2018 from 2017? Success: -3991.0
<< -$9,401-(-$5,410) || [-9401.

In [92]:
annotated_results = cga_utils.annotate_results(res)
cga_utils.calc_overall_em(annotated_results)

0.7142857142857143